In [1]:
import sys
sys.path.insert(0, "/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/")
import torch

In [2]:
from src.retrievers.e5 import E5Retriever

### E5-model test

In [3]:
retriever = E5Retriever()

Loading query E5-model...
Loading document E5-model...


In [7]:
a = retriever.model.embed_documents(["Hello world"])

In [10]:
len(a[0])

768

In [11]:
import numpy as np

In [22]:
((1/2)*np.log2(1/2)) + ((1/2)*np.log2(1/2))* (4/7)

-0.7857142857142857

### FiD-model test

In [ ]:
from src.fid_model import FiDT5
import transformers
import gc
gc.collect()

In [ ]:
model_name = 'google-t5/t5-base'
t5 = transformers.T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = transformers.T5Tokenizer.from_pretrained(model_name)

In [3]:
model = FiDT5(t5.config)
model.encoder.n_passages = 4

In [5]:
input_ids = torch.randint(0, 255, size=(2,2,512))
attention_mask = torch.randint(0, 255, size=(2,2,512))
labels = torch.randint(0, 255, size=(2,512))

In [ ]:
gen_out = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=10)

In [ ]:
forward_out = model(input_ids=input_ids,
          attention_mask=attention_mask,
          labels=labels)

### ColBERT test

In [4]:
import torch.nn.functional as F
import torch

In [43]:
B = 2
N = 3
Q_L = 2
D_L = 4
K = 3

In [48]:
q_batch = F.normalize(torch.randn(B,Q_L,K), dim=-1)
q_batch.requires_grad=True
q_mask = torch.randint(0,2,size=(B,Q_L)).bool()
d_batch = F.normalize(torch.randn(N,D_L,K), dim=-1)
d_batch.requires_grad=True
d_mask = torch.randint(0,2,size=(N,D_L)).bool()

print(q_batch.shape, d_batch.shape)
print(q_mask.shape, d_mask.shape)

torch.Size([2, 2, 3]) torch.Size([3, 4, 3])
torch.Size([2, 2]) torch.Size([3, 4])


In [49]:
batch_scores = torch.tensor([], requires_grad=True)
for i in range(q_batch.shape[0]):
    C = F.cosine_similarity(q_batch[i].unsqueeze(1).unsqueeze(1), d_batch, dim=-1)
    C = C.permute((1,0,2))
    print(C.shape)

    d_masked_C = C.masked_fill(d_mask.unsqueeze(1), -100)
    print(d_masked_C.shape)

    max_C = d_masked_C.max(dim=-1).values
    print(max_C.shape)

    q_masked_C = max_C.masked_fill(q_mask[i].unsqueeze(0), 0)
    print(q_masked_C.shape)

    scores = q_masked_C.sum(dim=-1)
    print(scores.shape)

    batch_scores = torch.cat((batch_scores, scores.unsqueeze(0)), dim=0)

torch.Size([3, 2, 4])
torch.Size([3, 2, 4])
torch.Size([3, 2])
torch.Size([3, 2])
torch.Size([3])
torch.Size([3, 2, 4])
torch.Size([3, 2, 4])
torch.Size([3, 2])
torch.Size([3, 2])
torch.Size([3])


In [50]:
batch_scores

tensor([[-1.3669, -0.5969, -0.8911],
        [-0.1125,  0.3626, -0.2524]], grad_fn=<CatBackward0>)

In [251]:
loss = batch_scores.sum()

In [ ]:
q_batch.grad

In [252]:
loss.backward()

In [ ]:
for b_i in range(q.shape[0]):
    batch_scores = []
    for doc_i in range(d.shape[1]):
        doc_scores = torch.tensor([
            (d[b_i][doc_i] - q[b_i][q_token_i]).pow(2).sum().sqrt() 
            for q_token_i in range(q.shape[1])], requires_grad=True)
        
        print(doc_scores.shape)
        print(d_mask[b_i][doc_i].shape)

        max_score = torch.masked_select(
            doc_scores, d_mask[b_i][doc_i]).max(0).values

        print(doc_scores)
        print(max_score)

In [26]:
q[0]

tensor([ 0.4106,  0.1958, -0.5337,  0.3719, -0.4051, -0.6870, -0.8315,  0.4174,
        -1.0673,  0.5532,  0.8065,  0.3561,  0.0629, -0.3530,  0.8857,  0.9097,
        -0.5392, -1.3984,  1.7648,  1.2462, -0.5269, -0.4240,  1.6104,  0.3616,
         0.6950,  1.5886, -0.4794,  0.2703, -0.8660, -0.8782,  0.5527,  1.4357])

In [24]:
d[0][0] - q[0]

tensor([ 0.3448, -2.2135,  0.2222, -1.6303,  0.2953,  1.2714,  0.4842, -0.9713,
         1.7690, -2.6323, -1.2720, -1.0291, -1.0879,  0.2451,  1.1824, -0.3049,
         0.4246,  1.2510, -1.0612, -2.6815, -0.0453,  1.1504, -2.3646,  1.5032,
        -2.3634, -2.1664,  0.5274,  1.0541,  1.1237,  1.6425, -0.1971, -2.2892])

In [25]:
d[0][0]

tensor([ 0.7554, -2.0177, -0.3115, -1.2583, -0.1098,  0.5844, -0.3473, -0.5539,
         0.7017, -2.0791, -0.4655, -0.6730, -1.0251, -0.1079,  2.0681,  0.6048,
        -0.1146, -0.1474,  0.7036, -1.4353, -0.5723,  0.7263, -0.7542,  1.8648,
        -1.6684, -0.5778,  0.0480,  1.3244,  0.2577,  0.7643,  0.3556, -0.8535])

In [22]:
q_un = q.unsqueeze(2)
d_un = d.unsqueeze(1)

print(q_un.shape, d_un.shape)

torch.Size([10, 32, 1]) torch.Size([4, 1, 10, 32])


In [23]:
q_un - d_un

RuntimeError: The size of tensor a (32) must match the size of tensor b (10) at non-singleton dimension 2